In [59]:
import requests
import pandas as pd
import json
import my_api_key as key


In [60]:
api_key = key.api_key

In [61]:
lat= 49.314695
lon= -2.833333
url = "https://api.openweathermap.org/data/3.0/onecall?lat={}&lon={}&appid={}".format(lat,lon,api_key)

In [62]:
df = pd.read_csv("gps_data.csv")
df.head()


,Ville,Latitude,Longitude
0,Mont Saint Michel,48.635954,-1.511460
1,Saint Malo,48.649518,-2.026041
2,Bayeux,49.276462,-0.702474
3,Le Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966


In [63]:
import csv

# Fonction pour récupérer la température et le champ "main" d'une ville
def get_temperature_and_main(latitude, longitude):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    if "current" in data:
        temperature_celsius = data["current"]["temp"] - 273.15  # Convertir de Kelvin en Celsius
        weather_main = data["current"]["weather"][0]["main"]
        return temperature_celsius, weather_main
    else:
        return None, None

csv_filename = "temperature.csv"

# Créer un fichier CSV
with open('temperature.csv', 'w', newline='') as csvfile:
    fieldnames = ['Ville', 'Température (°C)', 'Météo']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Parcourir les lignes du DataFrame
    for index, row in df.iterrows():
        ville = row['Ville']
        latitude = row['Latitude']
        longitude = row['Longitude']
        temperature, weather_main = get_temperature_and_main(latitude, longitude)
        if temperature is not None:
            temperature_rounded = round(temperature)  # Arrondir la température à l'entier le plus proche
            writer.writerow({'Ville': ville, 'Température (°C)': temperature_rounded, 'Météo': weather_main})
            print(f"Ville: {ville}, Température : {temperature_rounded}°C, Météo: {weather_main}")
        else:
            writer.writerow({'Ville': ville, 'Température (°C)': 'N/A', 'Météo': 'N/A'})

print("Les résultats ont été enregistrés dans 'temperature.csv'")


Ville: Mont Saint Michel, Température : 16°C, Météo principale: Rain
Ville: Saint Malo, Température : 16°C, Météo principale: Clouds
Ville: Bayeux, Température : 17°C, Météo principale: Clouds
Ville: Le Havre, Température : 16°C, Météo principale: Clouds
Ville: Rouen, Température : 17°C, Météo principale: Clouds
Ville: Paris, Température : 18°C, Météo principale: Clouds
Ville: Amiens, Température : 17°C, Météo principale: Clouds
Ville: Lille, Température : 16°C, Météo principale: Clouds
Ville: Strasbourg, Température : 15°C, Météo principale: Rain
Ville: Chateau du Haut Koenigsbourg, Température : 11°C, Météo principale: Clouds
Ville: Colmar, Température : 13°C, Météo principale: Clouds
Ville: Eguisheim, Température : 13°C, Météo principale: Clouds
Ville: Besancon, Température : 18°C, Météo principale: Clouds
Ville: Dijon, Température : 18°C, Météo principale: Clear
Ville: Annecy, Température : 18°C, Météo principale: Clouds
Ville: Grenoble, Température : 20°C, Météo principale: Clouds